# Basic Plotting with Bokeh

In this notebook we create some basic plots with `bokeh`, which come with a fair amount of interactivity out of the box.  

In the next notebook we will add some interactive refinements to our plots.

### Data

Let's begin by loading the data that we will need for our visualizations.

In [1]:
import pandas_datareader as pdr
import pandas as pd
import numpy as np

# SPY Close, Returns, Realized Volatility, and VIX
df_spy = pdr.get_data_yahoo(['SPY', '^VIX'], start='2016-01-01', end='2021-09-30')
df_spy = df_spy.round(2)
df_spy = df_spy['Close'].reset_index()
df_spy.rename(columns={'Date':'date','SPY':'close','^VIX':'vix'}, inplace=True)
df_spy['return'] = np.log(df_spy['close'] / df_spy['close'].shift(1))
df_spy['realized_vol'] = df_spy['return'].rolling(42).std() * np.sqrt(252)

# SPY Monthly Returns
df_spy['year'] = df_spy['date'].dt.year
df_spy['month'] = df_spy['date'].dt.month
df_monthly = df_spy.groupby(['year', 'month'], as_index=False)[['return']].sum()
df_monthly['year_month'] = (df_monthly['year'] * 100) + df_monthly['month']
df_monthly['year_month'] = df_monthly['year_month'].astype(str)

# Implied Leverage Effect
df_spy['vix_change'] = df_spy['vix'].diff()
df_spy['vix'] = df_spy['vix'] / 100
df_spy['vix_change'] = df_spy['vix_change'] / 100

# Asset Allocation
df_asset_allocation = pd.read_csv('asset_allocation.csv', parse_dates=['trade_date'])

### Two Interfaces

There are two interfaces for working with `bohek`:


`bokeh.models`: a low level interface that gives you complete control over how `bokeh` creates all elements of your visualization.

`bokeh.plotting`: a high level, general-purpose interface that is similar to plotting interfaces of libraries such as Matplotlib or Matlab.  It automatically assembles plots with default elements such as axes, grids, and tools for you.


Our focus will be on the `bokeh.plotting` interface.

### Blank Graph

Let's begin by creating a blank graph, and examine the elements of the code:

`bokeh.io.output_notebook()` - output the graph inline in the notebook.

`bokeh.plotting.figure()` - creates the basic plot object, called a `Figure` model.

`bokeh.plotting.show()` - outputs the `Figure` model.

In [2]:
# import bokeh functions
from bokeh.io import output_notebook
from bokeh.plotting import figure, show

# output inline
output_notebook()

# create figure() object with a title
fig = figure(title='Blank Figure')

# output to notebook
show(fig)

Loading BokehJS ...

Notice the toolbar on the right of the figure above which has a variety of interactive tools.  These aren't that interesting at the moment because our graph is blank.

### Basic Line Plot

Let's create our first proper graph by adding a `line` glyph that plot the SPY close prices.  

Notice that we also modify our `Figure` object by adding arguments to the `figure()` function.

Now the interactive tools are a bit more interesting.

In [3]:
# import bokeh functions
from bokeh.io import output_notebook, show
from bokeh.plotting import figure

# output inline
output_notebook()

# create figure() object with a title
p = figure(width=600, height=400, x_axis_type='datetime', title='SPY Close Price',
           x_axis_label='date', y_axis_label='close price')

# adding a line glyph to display close prices by passing data Series directly
p.line(x=df_spy['date'], y=df_spy['close'])

# output to notebook
show(p)

Loading BokehJS ...

### `ColumnDataSource`

In the example above, we passed our data to be graphed directly into the `p.line()` glyph by specifying particular columns of `df_spy`, which is a `pandas.DataFrame`.  If you recall, the columns of a `DataFrame` are `pandas.Series` objects.  We can also pass `lists` and `numpy.arrays` directly into a glyph.

However, it is more fruitful to use a `ColumnDataSource` object.  We can easily create one from a `DataFrame` called `df` with the following syntax: `ColumnDataSource(df)`.

In [4]:
# import bokeh functions
from bokeh.io import output_notebook, show
from bokeh.plotting import figure
from bokeh.models import ColumnDataSource

# output inline
output_notebook()

# creating ColumnDataSource from DataFrame
cds = ColumnDataSource(df_spy)

# creating figure() object
p = figure(width=600, height=400, x_axis_type='datetime', title='SPY Close Price',
           x_axis_label='date', y_axis_label='close price')

# adding a line glyph to display close prices, passing data via ColumnDataSource
p.line(x='date', y='close',source=cds)

# output to notebook
show(p)

Loading BokehJS ...

### Basic Scatter

Next, let's create a basic scatter plot using the `.circle()` glyph.  Notice that we have reformatted the x-axis and y-axis to be expressed as percents.

In [5]:
# import bokeh functions
from bokeh.io import output_notebook, show
from bokeh.plotting import figure
from bokeh.models import ColumnDataSource
from bokeh.models import NumeralTickFormatter

# output inline
output_notebook()

# creating a ColumnDataSource from DataFrame
cds = ColumnDataSource(df_spy)

# creating figure() object with a title
p = figure(plot_width=500, plot_height=500, title='SPY Implied Leverage Effect',
           x_axis_label='return', y_axis_label='vix change')

# adding circle glyph, passing data via ColumnDataSource
p.circle('return', 'vix_change', source=cds)

# formatting the x-axis and y-axis to percents
p.xaxis.formatter = NumeralTickFormatter(format='0%') 
p.yaxis.formatter = NumeralTickFormatter(format='0%')

# output to notebook           
show(p)

Loading BokehJS ...

### Other Glyphs

We can recreate this scatter plot with a larger hexes instead of circles.  We also modified the outline color and fill color, both of which were affected by arguments to `p.hex()`.

In [6]:
# import bokeh functions
from bokeh.io import output_notebook, show
from bokeh.plotting import figure
from bokeh.models import ColumnDataSource

# outpute inline
output_notebook()

# creating ColumnDataSource from DataFrame
cds = ColumnDataSource(df_spy)

# creating figure() object
p = figure(plot_width=600, plot_height=400, title='SPY Implied Leverage Effect',
           x_axis_label='return', y_axis_label='vix change')

# adding hex glyph, while changing some visual paramenters
p.hex('return', 'vix_change', line_color="navy", fill_color="orange", size=15, source=cds)

# formatting the x-axis and y-axis to percents
p.xaxis.formatter = NumeralTickFormatter(format='0%') 
p.yaxis.formatter = NumeralTickFormatter(format='0%')

show(p)

Loading BokehJS ...

### Glyphs Gallery

There are a wide variety of glyphs that can be used for scatter plots.  They are demonstrated below in a visual that I borrowed from the `bokeh` documentation.

In [7]:
from numpy.random import random
from bokeh.io import output_notebook
from bokeh.core.enums import MarkerType
from bokeh.plotting import figure, show


output_notebook()

p = figure(title="Bokeh Markers", toolbar_location=None)
p.grid.grid_line_color = None
p.background_fill_color = "#eeeeee"
p.axis.visible = False
p.y_range.flipped = True

N = 10

for i, marker in enumerate(MarkerType):
    x = i % 4
    y = (i // 4) * 4 + 1

    p.scatter(random(N)+2*x, random(N)+y, marker=marker, size=14,
              line_color="navy", fill_color="orange", alpha=0.5)

    p.text(2*x+0.5, y+2.5, text=[marker],
           text_color="firebrick", text_align="center", text_font_size="13px")

show(p)

Loading BokehJS ...

### Barchart with Monthly Returns

Here is a barchart of SPY monthly returns using the `p.vbar()` renderer.

In [8]:
# import bokeh functions
from bokeh.io import output_notebook
from bokeh.models import ColumnDataSource
from bokeh.models import NumeralTickFormatter
from bokeh.plotting import figure, show
from bokeh.plotting import reset_output
import math

# output inline
output_notebook()

# creating ColumnDataSource from DataFrame
source = ColumnDataSource(df_monthly)

# initializing the figure
p = figure(width=1000, height=400, x_range=df_monthly['year_month'], title='SPY Monthly Returns 2016Q1 to 2021Q3',
           x_axis_label='month', y_axis_label='monthly return')

# adding vbar glyphs, passing data with ColumnDataSource
p.vbar(x='year_month', bottom=0, top='return', color='blue', width=0.75, source=source)

# formatting the y-axis to percents
p.yaxis.formatter = NumeralTickFormatter(format='0%')

# rotating x-axis labels
p.xaxis.major_label_orientation = math.pi/4

# output graph
show(p)

Loading BokehJS ...

### Stacked Areas with Varying Asset Allocations

Next, we create a stacked area chart to visualize our asset allocations.  In order to get a useful set of colors, we need to use a palette from the `bokeh.palettes` module.

In [9]:
# import bokeh functions
import bokeh
from bokeh.models import ColumnDataSource
from bokeh.models import NumeralTickFormatter
from bokeh.plotting import figure, output_file, show
from bokeh.io import output_notebook
import pandas as pd

# output inline
output_notebook()

# putting data into ColumnDataSource object
source = ColumnDataSource(df_asset_allocation)

# initializing the figure
p = figure(width=900, height=500, x_axis_type='datetime', title='Asset Allocation 2019-2020',
           x_axis_label='month', y_axis_label='allocation')

# choosing assets and colors to graph
assets = df_asset_allocation.drop(columns='trade_date').columns
num_assets = len(assets)
colors = bokeh.palettes.magma(num_assets) # choosing color palette

# adding glyph with legend
p.varea_stack(assets, x='trade_date', color=colors, source=source, legend_label=assets.to_list())

# reversing order of legend to match order of graph
p.legend[0].items.reverse()

# reformatting y-axis as percent 
p.yaxis.formatter = NumeralTickFormatter(format='0%')

# moving legend off of the graph
p.add_layout(p.legend[0], 'right')

# output graph
show(p)

Loading BokehJS ...

### Layouts

Layout functions let you build a grid of plots and widgets. You can have as many rows, columns, or grids of plots in one layout as you like.

We will use the `column()` function to vertically stack our leverage effect line graphs.  Notice that each graph has its own set of interactive tools because they are not *linked* together.  We will remedy this in the next notebook.

In [10]:
# import bokeh functions
from bokeh.io import output_notebook, show
from bokeh.plotting import figure
from bokeh.layouts import column

# output inline
output_notebook()

# defining multiple plots
p1 = figure(width=600, height=200, x_axis_type='datetime', title='SPY Leverage Effecct')
p1.line(x=df_spy['date'], y=df_spy['close'])

p2 = figure(width=600, height=200, x_axis_type='datetime')
p2.line(x=df_spy['date'], y=df_spy['return'])
p2.yaxis.formatter = NumeralTickFormatter(format='0%')

p3 = figure(width=600, height=200, x_axis_type='datetime')
p3.line(x=df_spy['date'], y=df_spy['realized_vol'])
p3.yaxis.formatter = NumeralTickFormatter(format='0%')

p4 = figure(width=600, height=200, x_axis_type='datetime')
p4.line(x=df_spy['date'], y=df_spy['vix'])
p4.yaxis.formatter = NumeralTickFormatter(format='0%')

# putting them together using column()
show(column(p1, p2, p3, p4))

Loading BokehJS ...